In [1]:
import torch
vocab_size=50257
output_dim=256
token_embedding_layer=torch.nn.Embedding(vocab_size,output_dim)

In [2]:
from torch.utils.data import Dataset,DataLoader
class GPTdataset(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]
        
        ids=tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
        
        for i in range(0,len(ids)-max_length,stride):
            input_chunk=ids[i:i+max_length]
            target_chunk=ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]
        

In [3]:
def gpt_data_loader(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
   
    tokenizer = tiktoken.get_encoding("gpt2")
    

    dataset = GPTdataset(txt, tokenizer, max_length, stride)
    
   
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return data_loader
    

In [4]:
import importlib
import tiktoken


In [5]:
tokenizer=tiktoken.get_encoding("gpt2")

In [6]:
with open("Shikhar_Goel_Resume.txt","r",encoding="utf-8") as f:
    raw_text=f.read()
encoded_text=tokenizer.encode(raw_text)
print(len(raw_text))

1547


In [9]:
import torch
data_loader=gpt_data_loader(raw_text,batch_size=8,max_length=4,stride=4,shuffle=False)
data_iter=iter(data_loader)
inputs,target=next(data_iter)

In [10]:
vector_embeddings=token_embedding_layer(inputs)

In [13]:
vector_embeddings.size()

torch.Size([8, 4, 256])

In [18]:
context_length=max_length=4
pos_embedding_layer=torch.nn.Embedding(context_length,output_dim)

In [19]:
pos_embedding=pos_embedding_layer(torch.arange(max_length))

In [23]:
final_embedding=vector_embeddings+pos_embedding
final_embedding.size()

torch.Size([8, 4, 256])